Wikipedia Recommendation system

-----

Authors:
- Martyna Stasiak id.156071

The aim of this project is to generate the recommendations on wikipedia articles basing on the ones that user have liked. <br>
To do that we have used 10 000 initial articles that were obtained by web crawling, starting from the https://en.wikipedia.org/wiki/Machine_learning article; later they were saved in the csv file, so if there is a need the file working as our database might be changed.

------

Libraries that we have used and are necessary for this project:

In [130]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, wordpunct_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

from IPython.core.display import display, HTML

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mmart\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mmart\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mmart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\mmart\AppData\Local\Temp\ipykernel_30212\1868813398.py:21: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Crawling and saving our articles

In this part we create the file that will work as or database containing all possible wikipedia articles. <br>
We perform the crawling by ...... <explain precisely> <br>


In [131]:
def crawlArticles(start_url, max_articles):
    visited = set()
    to_visit = [start_url]
    articles = []
    
    while to_visit and len(articles) < max_articles:
        page = to_visit.pop(0)
        if page in visited:
            continue
        visited.add(page)
        
        try:
            response = requests.get(page)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            title = soup.find('h1').text # article's title
            paragraphs = soup.find_all('p') # article's paragraphs
            content = ' '.join([p.text for p in paragraphs]) # article's content that is inside paragraphs
            articles.append({"title": title, "link": page, "content": content})
            
            # extracting and filtering new links
            for link in soup.find_all('a', href=True): # we look for all links in the page
                href = link['href']
                if href.startswith('/wiki/') and ':' not in href and '#' not in href and 'Main_Page' not in href:
                    full_url = "https://en.wikipedia.org" + href
                    if full_url not in visited:
                        to_visit.append(full_url)
            time.sleep(0.5) # be polite to Wikipedia
        except:
            pass
        
    return articles

In [132]:
articles = crawlArticles("https://en.wikipedia.org/wiki/Machine_learning", 10)
df = pd.DataFrame(articles)

In [133]:
def saveDatabase(df, fileName):
    df.to_csv(fileName, index=False)
    return None

In [134]:
saveDatabase(df, 'articles.csv')

In [135]:
df.head()

,title,link,content
0,Machine learning,https://en.wikipedia.org/wiki/Machine_learning,Machine learning (ML) is a field of study in a...
1,Machine Learning (journal),https://en.wikipedia.org/wiki/Machine_Learning...,Machine Learning is a peer-reviewed scientifi...
2,Statistical learning in language acquisition,https://en.wikipedia.org/wiki/Statistical_lear...,Statistical learning is the ability for humans...
3,Data mining,https://en.wikipedia.org/wiki/Data_mining,Data mining is the process of extracting and d...
4,Supervised learning,https://en.wikipedia.org/wiki/Supervised_learning,"In machine learning, supervised learning (SL) ..."


### Text preprocessing

Now when we have the database containing the articles we have to do the preprocessing; <br>
we have done: 
- lemmatization
- deleting the stopwords
- 

In [136]:
stopWords = set(stopwords.words('english'))
porter = PorterStemmer()
lancaster = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [137]:
def preprocessArticles(df, tokenizer=word_tokenize, stemmer=None, lemmatizer=None, useLemmatizer=False):
    tokens = tokenizer(df['content'].lower())
    terms = [word for word in tokens if word.isalpha() and word not in stopWords] # remove stopwords and non-alphabetic words
    if stemmer:
        processed = [stemmer.stem(word) for word in terms]
    elif useLemmatizer and lemmatizer:
        processed = [lemmatizer.lemmatize(word) for word in terms]
    else:
        processed = terms
    return ' '.join(processed)
    

In [138]:
# Define preprocessing variations
variations_wordtokenizer = {
    "porter_stemmer": lambda row: preprocessArticles(row, tokenizer=word_tokenize, stemmer=porter),
    "lancaster_stemmer": lambda row: preprocessArticles(row, tokenizer=word_tokenize, stemmer=lancaster),
    "lemmatization": lambda row: preprocessArticles(row, tokenizer=word_tokenize, lemmatizer=lemmatizer, useLemmatizer=True)
}

# Apply variations without modifying the original DataFrame
results_wordtokenizer = pd.DataFrame({
    "title": df["title"],
    "original_content": df["content"]
})

for name, preprocess_function in variations_wordtokenizer.items():
    # Apply each variation to the content column using the original function
    results_wordtokenizer[name] = df.apply(preprocess_function, axis=1)


variations_wordpunct = {
    "porter_stemmer": lambda row: preprocessArticles(row, tokenizer=word_tokenize, stemmer=porter),
    "lancaster_stemmer": lambda row: preprocessArticles(row, tokenizer=word_tokenize, stemmer=lancaster),
    "lemmatization": lambda row: preprocessArticles(row, tokenizer=word_tokenize, lemmatizer=lemmatizer, useLemmatizer=True)
}

# Apply variations without modifying the original DataFrame
results_wordpunct = pd.DataFrame({
    "title": df["title"],
    "original_content": df["content"]
})

for name, preprocess_function in variations_wordpunct.items():
    # Apply each variation to the content column using the original function
    results_wordpunct[name] = df.apply(preprocess_function, axis=1)


In [139]:
print("Word Tokenizer")
columns_to_display = ["title", "original_content"] + list(variations_wordtokenizer.keys())
results_wordtokenizer[columns_to_display].head()

Word Tokenizer


,title,original_content,porter_stemmer,lancaster_stemmer,lemmatization
0,Machine learning,Machine learning (ML) is a field of study in a...,machin learn ml field studi artifici intellig ...,machin learn ml field study art intellig conce...,machine learning ml field study artificial int...
1,Machine Learning (journal),Machine Learning is a peer-reviewed scientifi...,machin learn scientif journal publish sinc for...,machin learn sci journ publ sint forty edit me...,machine learning scientific journal published ...
2,Statistical learning in language acquisition,Statistical learning is the ability for humans...,statist learn abil human anim extract statist ...,stat learn abl hum anim extract stat regul wor...,statistical learning ability human animal extr...
3,Data mining,Data mining is the process of extracting and d...,data mine process extract discov pattern larg ...,dat min process extract discov pattern larg da...,data mining process extracting discovering pat...
4,Supervised learning,"In machine learning, supervised learning (SL) ...",machin learn supervis learn sl paradigm model ...,machin learn superv learn sl paradigm model tr...,machine learning supervised learning sl paradi...


In [140]:
print("Word Punct Tokenizer")
columns_to_display = ["title", "original_content"] + list(variations_wordpunct.keys())
results_wordpunct[columns_to_display].head()

Word Punct Tokenizer


,title,original_content,porter_stemmer,lancaster_stemmer,lemmatization
0,Machine learning,Machine learning (ML) is a field of study in a...,machin learn ml field studi artifici intellig ...,machin learn ml field study art intellig conce...,machine learning ml field study artificial int...
1,Machine Learning (journal),Machine Learning is a peer-reviewed scientifi...,machin learn scientif journal publish sinc for...,machin learn sci journ publ sint forty edit me...,machine learning scientific journal published ...
2,Statistical learning in language acquisition,Statistical learning is the ability for humans...,statist learn abil human anim extract statist ...,stat learn abl hum anim extract stat regul wor...,statistical learning ability human animal extr...
3,Data mining,Data mining is the process of extracting and d...,data mine process extract discov pattern larg ...,dat min process extract discov pattern larg da...,data mining process extracting discovering pat...
4,Supervised learning,"In machine learning, supervised learning (SL) ...",machin learn supervis learn sl paradigm model ...,machin learn superv learn sl paradigm model tr...,machine learning supervised learning sl paradi...


In [141]:
df['processedContent'] = df.apply(lambda row: preprocessArticles(row, tokenizer=word_tokenize, stemmer=None, 
                                                          lemmatizer=lemmatizer, useLemmatizer=True), axis=1)
saveDatabase(df, 'processed_articles.csv')

columnstoUse = ['title', 'content','processedContent']
df[columnstoUse].head()

,title,content,processedContent
0,Machine learning,Machine learning (ML) is a field of study in a...,machine learning ml field study artificial int...
1,Machine Learning (journal),Machine Learning is a peer-reviewed scientifi...,machine learning scientific journal published ...
2,Statistical learning in language acquisition,Statistical learning is the ability for humans...,statistical learning ability human animal extr...
3,Data mining,Data mining is the process of extracting and d...,data mining process extracting discovering pat...
4,Supervised learning,"In machine learning, supervised learning (SL) ...",machine learning supervised learning sl paradi...


### TF-IDF and Cosine Similarity

In [142]:
def tf_idf(df):
    tfidf = TfidfVectorizer(use_idf=True, smooth_idf=False)
    tfidf_matrix = tfidf.fit_transform(df['processedContent'])

    dfTFIDF = pd.DataFrame(tfidf_matrix.toarray(), index=df['title'], columns=tfidf.get_feature_names_out())
    return tfidf, tfidf_matrix, dfTFIDF


In [143]:
tfidf, tfidf_matrix, dfTFIDF = tf_idf(df)

dfTFIDF.head()

,aahc,abandon,abandoned,ability,able,abnormal,absence,abstract,abstracting,abstraction,...,yet,yield,yielded,yim,yoshua,young,youtube,yu,zero,zip
title,,,,,,,,,,,,,,,,,,,,,
Machine learning,0.000000,0.0,0.00762,0.007813,0.007813,0.00762,0.005085,0.005085,0.000000,0.000000,...,0.015256,0.005085,0.006021,0.000000,0.0,0.000000,0.0,0.000000,0.005085,0.015241
Machine Learning (journal),0.000000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
Statistical learning in language acquisition,0.000000,0.0,0.00000,0.050090,0.116877,0.00000,0.000000,0.005434,0.008142,0.008142,...,0.005434,0.000000,0.000000,0.008142,0.0,0.048852,0.0,0.024426,0.000000,0.000000
Data mining,0.013261,0.0,0.00000,0.006799,0.013597,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
Supervised learning,0.000000,0.0,0.00000,0.000000,0.051744,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [149]:
def recommendArticles(history, df, top_n=5):
    tfidf, tfidf_matrix, dfTFIDF = tf_idf(df)
    df = df.set_index('title')
    
    historyContent = ' '.join(df.loc[history,'processedContent'])
    historyVector = tfidf.transform([historyContent]).toarray()[0]
    
    cosineDistance = dfTFIDF.apply(lambda row: cosine(row, historyVector), axis=1)
    similarityScores = 1 - cosineDistance
    
    recommendations = pd.DataFrame({
        'title': dfTFIDF.index, 
        'link': df.loc[dfTFIDF.index, 'link'],
        'similarity': similarityScores
    })
    
    # we exclude the articles that user have already seen from the recommendations
    recommendations = recommendations[~recommendations['title'].isin(history)]
    
    recommendations = recommendations.sort_values(by='similarity', ascending=False).reset_index(drop=True)
    
    recommendations = recommendations.head(top_n)

    
    recommendations = HTML(recommendations.to_html(render_links=True, escape=False))
    
    return recommendations

In [153]:
history = ['Machine learning', 'Supervised learning']
recommendations = recommendArticles(history, df, top_n=5)
recommendations

,title,link,similarity
0,Weak supervision,https://en.wikipedia.org/wiki/Semi-supervised_learning,0.481517
1,Self-supervised learning,https://en.wikipedia.org/wiki/Self-supervised_learning,0.480971
2,Meta-learning (computer science),https://en.wikipedia.org/wiki/Meta-learning_(computer_science),0.472754
3,Unsupervised learning,https://en.wikipedia.org/wiki/Unsupervised_learning,0.460006
4,Reinforcement learning,https://en.wikipedia.org/wiki/Reinforcement_learning,0.369354


In [ ]:
recommendations